<a href="https://colab.research.google.com/github/takky0330/wisper/blob/main/InsightFace_mesen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!ls -d /usr/local/cuda-*
!nvcc --version

/usr/local/cuda-12  /usr/local/cuda-12.2
nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0


In [2]:
!wget https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64/cuda-ubuntu1804.pin
!sudo mv cuda-ubuntu1804.pin /etc/apt/preferences.d/cuda-repository-pin-600
!wget https://developer.download.nvidia.com/compute/cuda/11.8.0/local_installers/cuda-repo-ubuntu1804-11-8-local_11.8.0-520.61.05-1_amd64.deb
!sudo dpkg -i cuda-repo-ubuntu1804-11-8-local_11.8.0-520.61.05-1_amd64.deb
!sudo cp /var/cuda-repo-ubuntu1804-11-8-local/cuda-*-keyring.gpg /usr/share/keyrings/
!sudo apt-get update
!sudo apt-get -y install cuda-11-8

--2024-10-23 08:53:32--  https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64/cuda-ubuntu1804.pin
Resolving developer.download.nvidia.com (developer.download.nvidia.com)... 152.195.19.142
Connecting to developer.download.nvidia.com (developer.download.nvidia.com)|152.195.19.142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 190 [application/octet-stream]
Saving to: ‘cuda-ubuntu1804.pin’

cuda-ubuntu1804.pin 100%[===================>]     190  --.-KB/s    in 0s      

2024-10-23 08:53:32 (6.33 MB/s) - ‘cuda-ubuntu1804.pin’ saved [190/190]

--2024-10-23 08:53:32--  https://developer.download.nvidia.com/compute/cuda/11.8.0/local_installers/cuda-repo-ubuntu1804-11-8-local_11.8.0-520.61.05-1_amd64.deb
Resolving developer.download.nvidia.com (developer.download.nvidia.com)... 152.195.19.142
Connecting to developer.download.nvidia.com (developer.download.nvidia.com)|152.195.19.142|:443... connected.
HTTP request sent, awaiting response... 200 O

In [3]:
!pip install insightface

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 439.5/439.5 kB 8.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 106.2 MB/s eta 0:00:00
  Created wheel for insightface: filename=insightface-0.7.3-cp310-cp310-linux_x86_64.whl size=1055395 sha256=701052737ef88db920e5540db27b84287e96fef5f75ff5875049fa9cbad9f8ce
  Stored in directory: /root/.cache/pip/wheels/e3/d0/80/e3773fb8b6d1cca87ea1d33d9b1f20a223a6493c896da249b5
Successfully built insightface


In [4]:
!pip install onnxruntime-gpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.2/226.2 MB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 9.3 MB/s eta 0:00:00


In [5]:
from insightface.app import FaceAnalysis

/usr/local/lib/python3.10/dist-packages/albumentations/__init__.py:13: UserWarning: A new version of Albumentations is available: 1.4.18 (you have 1.4.15). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


In [6]:
!pip install moviepy

In [7]:
!pip install numpy==1.23.5              #最新VerではInseight実行時にエラーが出るためこのバージョンに下げる。このVerでは初回Import時に失敗する。 ランタイムの再起動も発生。

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 101.5 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albucore 0.0.16 requires numpy>=1.24, but you have numpy 1.23.5 which is incompatible.
albumentations 1.4.15 requires numpy>=1.24.4, but you have numpy 1.23.5 which is incompatible.
bigframes 1.22.0 requires numpy>=1.24.0, but you have numpy 1.23.5 which is incompatible.
chex 0.1.87 requires numpy>=1.24.1, but you have numpy 1.23.5 which is incompatible.
jax 0.4.33 requires numpy>=1.24, but you have numpy 1.23.5 which is incompatible.
jaxlib 0.4.33 requires numpy>=1.24, but you have numpy 1.23.5 which is incompatible.
xarray 2024.9.0 requires numpy>=1.24, but you have numpy 1.23.5 which 

In [1]:
!ls -d /usr/local/cuda-*
!nvcc --version

/usr/local/cuda-11  /usr/local/cuda-11.8  /usr/local/cuda-12  /usr/local/cuda-12.2
nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0


In [2]:
import os
p = os.getenv('PATH')
ld = os.getenv('LD_LIBRARY_PATH')
os.environ['PATH'] = f"/usr/local/cuda-11.8/bin:{p}"
os.environ['LD_LIBRARY_PATH'] = f"/usr/local/cuda-11.8/lib64:{ld}"

In [3]:
!ls -d /usr/local/cuda-*
!nvcc --version

/usr/local/cuda-11  /usr/local/cuda-11.8  /usr/local/cuda-12  /usr/local/cuda-12.2
nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0


In [4]:
!cat /usr/include/cudnn_version.h | grep CUDNN_MAJOR -A 2

#define CUDNN_MAJOR 8
#define CUDNN_MINOR 9
#define CUDNN_PATCHLEVEL 6
--
#define CUDNN_VERSION (CUDNN_MAJOR * 1000 + CUDNN_MINOR * 100 + CUDNN_PATCHLEVEL)

/* cannot use constexpr here since this is a C-only file */


In [5]:
import torch
print(torch.cuda.is_available())
print(torch.cuda.get_device_name())

True
NVIDIA A100-SXM4-40GB


In [6]:
import cv2
import numpy as np
from tqdm import tqdm
from insightface.app import FaceAnalysis                      #もう一度インポート
from moviepy.editor import VideoFileClip, AudioFileClip

/usr/local/lib/python3.10/dist-packages/albumentations/__init__.py:13: UserWarning: A new version of Albumentations is available: 1.4.18 (you have 1.4.15). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()
  if event.key is 'enter':



In [7]:
class INSIGHTFACE(object):
    def __init__(self, ctx_id=0, det_size_x=640, det_size_y=640):
        #self.app = FaceAnalysis(root='./.insightface')
        self.app = FaceAnalysis(root='./.insightface', providers=['CUDAExecutionProvider'])
        self.app.prepare(ctx_id=ctx_id, det_size=(det_size_x, det_size_y))
        self.faces = []

    def get_InsightFace(self, img):
        self.faces = self.app.get(np.asarray(img))
        #for face in faces:
        #    kps   = face.kps.astype(np.int)
        #    print(kps)
        rimg  = self.app.draw_on(img, self.faces)
        #cv2.imwrite("./output.jpg", rimg)
        return img, rimg, self.faces

    def draw_mesen(self, img):
        KPS = []
        for face in self.faces:
            kps = face.kps.astype(np.int)
            KPS.append(kps)
        EYE = _EYE = [[]] * 2
        out_img = img
        for _eye in KPS:
            EYE[0] = _eye[0]
            EYE[1] = _eye[1]
            lineSize  = abs(np.linalg.norm(np.array(EYE[0])-np.array(EYE[1])))
            lineWidth = int(lineSize / 4)
            if lineWidth <= 10:
                lineWidth = 10
            lineAngle = 1
            if EYE[1][0]-EYE[0][0] != 0:
                lineAngle   = (EYE[1][1]-EYE[0][1])/(EYE[1][0]-EYE[0][0])
            _EYE[0][0] = EYE[0][0] - (lineSize / 3)
            _EYE[0][1] = lineAngle * (_EYE[0][0] - EYE[0][0]) + EYE[0][1]
            _EYE[1][0] = EYE[1][0] + (lineSize / 3)
            _EYE[1][1] = lineAngle * (_EYE[1][0] - EYE[0][0]) + EYE[0][1]
            out_img = cv2.line(out_img,
                              (_EYE[0][0], _EYE[0][1]),
                              (_EYE[1][0], _EYE[1][1]),
                              (0,0,0),
                              thickness=lineWidth)
        return out_img

In [8]:
IF = INSIGHTFACE()

download_path: ./.insightface/models/buffalo_l


100%|██████████| 281857/281857 [00:03<00:00, 91722.91KB/s]


Applied providers: ['CUDAExecutionProvider', 'CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}, 'CUDAExecutionProvider': {'sdpa_kernel': '0', 'use_tf32': '1', 'prefer_nhwc': '0', 'tunable_op_max_tuning_duration_ms': '0', 'enable_skip_layer_norm_strict_mode': '0', 'tunable_op_tuning_enable': '0', 'tunable_op_enable': '0', 'use_ep_level_unified_stream': '0', 'device_id': '0', 'has_user_compute_stream': '0', 'gpu_external_empty_cache': '0', 'cudnn_conv_algo_search': 'EXHAUSTIVE', 'cudnn_conv1d_pad_to_nc1d': '0', 'gpu_mem_limit': '18446744073709551615', 'gpu_external_alloc': '0', 'gpu_external_free': '0', 'arena_extend_strategy': 'kNextPowerOfTwo', 'do_copy_in_default_stream': '1', 'enable_cuda_graph': '0', 'user_compute_stream': '0', 'cudnn_conv_use_max_workspace': '1'}}
find model: ./.insightface/models/buffalo_l/1k3d68.onnx landmark_3d_68 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CUDAExecutionProvider', 'CPUExecutionProvider'], with options: {'CPUExecutionProvi

In [9]:
import onnxruntime as ort

print(ort.get_device())
print(ort.get_available_providers())

provider = ['CUDAExecutionProvider','CPUExecutionProvider']
provider = ['CUDAExecutionProvider']
ort_sess = ort.InferenceSession('./.insightface/models/buffalo_l/w600k_r50.onnx', providers=provider)
print(ort_sess.get_providers())

GPU
['TensorrtExecutionProvider', 'CUDAExecutionProvider', 'CPUExecutionProvider']
['CUDAExecutionProvider', 'CPUExecutionProvider']


In [10]:
!cp ./drive/MyDrive/zoom_480p.mov ./
!cp ./drive/MyDrive/InterviewZero_0.mp4 ./
!cp ./drive/MyDrive/InterviewZero_1.mp4 ./
!cp ./drive/MyDrive/InterviewZero_2.mp4 ./

In [11]:
#video_name = "zoom_480p.mov"
video_name = "InterviewZero_0.mp4"
#video_name = "InterviewZero_1.mp4"
#video_name = "InterviewZero_2.mp4"
video_path = "./" + video_name
mask_path  = '.'.join(video_path.split('.')[:-1]) + '_mask.mp4'
audio_path = '.'.join(video_path.split('.')[:-1]) + '.wav'
output_path = '.'.join(video_path.split('.')[:-1]) + '_audio_mask.mp4'
video = VideoFileClip(video_path)
video.audio.write_audiofile(audio_path, ffmpeg_params=["-ac", "1"])  #モノラルで出力

MoviePy - Writing audio in ./InterviewZero_0.wav


MoviePy - Done.


In [12]:
cap = cv2.VideoCapture(video_path)
if cap.isOpened() is False:
    cap = None

In [13]:
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = float(cap.get(cv2.CAP_PROP_FPS))
frameNum = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

In [14]:
codec = cv2.VideoWriter_fourcc(*'mp4v')
writer = cv2.VideoWriter(mask_path, codec, fps, (width, height), True)

In [ ]:
bar = tqdm(total=frameNum, dynamic_ncols=True)
cap.set(cv2.CAP_PROP_POS_FRAMES, 1)
ret, img = cap.read()
while ret :
    input_img, draw_img, faces = IF.get_InsightFace(img)
    out_img = IF.draw_mesen(input_img)
    writer.write(out_img)
    ret, img = cap.read()
    bar.update(1)
writer.release()

  0%|          | 0/28781 [00:00<?, ?it/s]WARNING:py.warnings:/usr/local/lib/python3.10/dist-packages/insightface/utils/transform.py:68: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  P = np.linalg.lstsq(X_homo, Y)[0].T # Affine matrix. 3 x 4

Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  kps = face.kps.astype(np.int)

  6%|▌         | 1694/28781 [01:23<20:46, 21.73it/s]

In [ ]:
video_clip = VideoFileClip(mask_path)
audio_clip = AudioFileClip(audio_path)
video_clip = video_clip.set_audio(audio_clip)
video_clip.write_videofile(output_path, audio_codec = 'aac')

In [ ]:
! ls -alh

In [ ]:
!cp ./zoom_480p_mask.mp4 ./drive/MyDrive
!cp ./zoom_480p.mp3 ./drive/MyDrive
!cp ./zoom_480p_audio_mask.mp4 ./drive/MyDrive